<a href="https://colab.research.google.com/github/fsicardir/tp1_modelos/blob/main/tp1_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import requests
import random
from google.colab import files

In [2]:
def add_incompat(incompatibilities, name1, name2):
  incomp = incompatibilities.get(name1, set()) | {name2}
  incompatibilities[name1] = incomp

def add_time(times, name, time):
  times[name] = int(time)

In [3]:
def parse_file(lines):
  incompatibilities = {}
  times = {}

  for line in lines:
    fields = line.split()
    if fields[0] == 'e':
      add_incompat(incompatibilities, fields[1], fields[2])
      add_incompat(incompatibilities, fields[2], fields[1])
    if fields[0] == 'n':
      add_time(times, fields[1], fields[2])

  return times, incompatibilities
    

In [4]:

class Cloth:
  def __init__(self, name, time, incompatibles):
    self.name = name
    self.time = time
    self.incompatibles = incompatibles

  def __str__(self):
    return f'Cloth({self.name}, {self.time}, {self.incompatibles})'

  def __repr__(self):
    return self.__str__()


In [5]:
def create_clothes(times, incompats):
  clothes = []
  for name, time in times.items():
    clothes.append(Cloth(name, time, incompats.get(name, set())))
  return clothes

In [6]:
def is_compatible(cloth, batch):
  for c in batch:
    if c.name in cloth.incompatibles:
      return False
  return True

def _get_compatible_batch(cloth, batches, non_compatible):
  if len(batches) == 0:
    for elem in non_compatible:
      heapq.heappush(batches, elem)
    return 0, 0, set()
  order, batch_time, batch = heapq.heappop(batches)
  if is_compatible(cloth, batch):
    for elem in non_compatible:
      heapq.heappush(batches, elem)
    return order, batch_time, batch
  non_compatible.append((order, batch_time, batch))
  return _get_compatible_batch(cloth, batches, non_compatible)

def get_compatible_batch(cloth, batches):
  return _get_compatible_batch(cloth, batches, [])

In [7]:
def process(clothes):
  batches = []
  for cloth in clothes:
    order, batch_time, batch = get_compatible_batch(cloth, batches)
    if cloth.time > batch_time:
      batch_time = cloth.time
    batch.add(cloth)
    heapq.heappush(batches, (-batch_time, batch_time, batch))
  return batches
    

In [8]:
lines = requests.get("https://modelosuno.okapii.com/Problemas/primer_problema.txt").text.split('\n')

times, incompats = parse_file(lines)

clothes = create_clothes(times, incompats)

In [9]:
batches = process(clothes)

print(batches)

[(-10, 10, {Cloth(12, 9, {'11', '15', '5', '20', '13', '9', '16', '4', '10', '14', '19', '3'}), Cloth(17, 10, {'10', '1', '20', '18', '9', '19', '3'}), Cloth(2, 10, {'11', '5', '20', '1', '7', '8', '9', '4', '10', '6', '18', '3'})}), (-10, 10, {Cloth(7, 10, {'15', '5', '1', '8', '16', '4', '6', '2', '14', '3'}), Cloth(19, 4, {'11', '5', '1', '20', '12', '10', '17', '18', '3'}), Cloth(9, 9, {'11', '15', '1', '13', '12', '16', '10', '17', '2', '14', '18'})}), (-9, 9, {Cloth(10, 8, {'11', '15', '1', '13', '12', '9', '16', '17', '2', '14', '18', '19', '3'}), Cloth(6, 9, {'5', '1', '20', '13', '7', '8', '14', '4', '2', '15', '3'})}), (-9, 9, {Cloth(3, 9, {'11', '5', '1', '7', '8', '12', '4', '10', '17', '6', '2', '19'}), Cloth(13, 5, {'11', '15', '5', '20', '12', '9', '4', '10', '16', '6', '14'}), Cloth(18, 4, {'11', '20', '9', '4', '10', '17', '2', '19'})}), (-8, 8, {Cloth(11, 6, {'15', '20', '13', '12', '9', '4', '10', '16', '2', '14', '18', '19', '3'}), Cloth(1, 8, {'5', '7', '8', '9', '

In [10]:
def results(batches):
  acc = 0
  for order, time, clothes in batches:
    acc += time
  return len(batches), acc

In [11]:
num, total_time = results(batches)
print(f'number of batches: {num}')
print(f'total time: {total_time}')

number of batches: 9
total time: 69


In [12]:
min_time = float('Inf')
best_order = []
for i in range(10000):
  s = random.sample(clothes, len(clothes))
  b = process(s)
  num, total_time = results(b)
  if total_time < min_time:
    min_time = total_time
    best_order = b

print(f'number of batches: {len(best_order)}')
print(f'total time: {min_time}')

number of batches: 8
total time: 61


In [13]:
def write_solution(batches, filename):
  with open(filename, 'w') as file:
    i = 1
    for batch in batches:
      for cloth in batch[-1]:
        file.write(f'{cloth.name} {i}\n')
      i += 1


In [14]:
filename = '/entrega_1.txt'
write_solution(best_order, filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
files.download(filename)